<a href="https://colab.research.google.com/github/PhanindraDogiparti/Business-performance-Analytics/blob/main/Business_Performance_using_pyspark_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from pyspark.sql.functions import col,lit,when
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression
from pyspark.ml.regression import RandomForestRegressor
spark=SparkSession.builder.appName('MySession').getOrCreate()

In [5]:
pip install pyspark

In [7]:
df=spark.read.csv("/content/Business performance analytics.csv",header=True,inferSchema=True)

In [8]:
df.columns
df=df.na.drop()
df.columns

['rowid',
 'orderid',
 'orderdata',
 'shipdate',
 'shipmode',
 'customerid',
 'customername',
 'segment',
 'country',
 'city',
 'state',
 'postalcode',
 'region',
 'productid',
 'category',
 'subcategory',
 'productname',
 'sales',
 'quantity',
 'discount',
 'profit']

In [9]:
selection_df = df.select('region', 'subcategory', 'category', 'quantity', 'sales')
selection_df.printSchema()

root
 |-- region: string (nullable = true)
 |-- subcategory: string (nullable = true)
 |-- category: string (nullable = true)
 |-- quantity: string (nullable = true)
 |-- sales: string (nullable = true)



In [10]:
selection_df = selection_df.withColumn('sales', col('sales').cast('double'))
selection_df = selection_df.withColumn('quantity', col('quantity').cast('double'))

In [11]:
selection_df.show()

+-------+-----------+---------------+--------+--------+
| region|subcategory|       category|quantity|   sales|
+-------+-----------+---------------+--------+--------+
|  South|  Bookcases|      Furniture|     2.0|  261.96|
|  South|     Chairs|      Furniture|     3.0|  731.94|
|   West|     Labels|Office Supplies|     2.0|   14.62|
|  South|     Tables|      Furniture|     5.0|957.5775|
|  South|    Storage|Office Supplies|     2.0|  22.368|
|   West|Furnishings|      Furniture|     7.0|   48.86|
|   West|        Art|Office Supplies|     4.0|    7.28|
|   West|     Phones|     Technology|     6.0| 907.152|
|   West|    Binders|Office Supplies|     3.0|  18.504|
|   West| Appliances|Office Supplies|     5.0|   114.9|
|   West|     Tables|      Furniture|     9.0|1706.184|
|   West|     Phones|     Technology|     4.0| 911.424|
|  South|      Paper|Office Supplies|     3.0|  15.552|
|   West|    Binders|Office Supplies|     3.0| 407.976|
|Central| Appliances|Office Supplies|     5.0|  

In [12]:
selection_df = selection_df.na.drop()

indexer = StringIndexer (inputCols=["region", "subcategory"], outputCols=["region_indexed", "sub-Category_indexed"])
df_i = indexer.fit(selection_df).transform(selection_df)

df_i.show()


+-------+-----------+---------------+--------+--------+--------------+--------------------+
| region|subcategory|       category|quantity|   sales|region_indexed|sub-Category_indexed|
+-------+-----------+---------------+--------+--------+--------------+--------------------+
|  South|  Bookcases|      Furniture|     2.0|  261.96|           3.0|                12.0|
|  South|     Chairs|      Furniture|     3.0|  731.94|           3.0|                 7.0|
|   West|     Labels|Office Supplies|     2.0|   14.62|           0.0|                 9.0|
|  South|     Tables|      Furniture|     5.0|957.5775|           3.0|                10.0|
|  South|    Storage|Office Supplies|     2.0|  22.368|           3.0|                 4.0|
|   West|Furnishings|      Furniture|     7.0|   48.86|           0.0|                 2.0|
|   West|        Art|Office Supplies|     4.0|    7.28|           0.0|                 5.0|
|   West|     Phones|     Technology|     6.0| 907.152|           0.0|          

In [13]:
fa= VectorAssembler (inputCols=['region_indexed', 'sub-Category_indexed', 'quantity', 'sales' ], outputCol="Independent Features")
output = fa.transform(df_i)

In [14]:
final_data = output.select("Independent Features", "sales")
final_data.na.drop()
final_data.show()

+--------------------+--------+
|Independent Features|   sales|
+--------------------+--------+
|[3.0,12.0,2.0,261...|  261.96|
|[3.0,7.0,3.0,731.94]|  731.94|
| [0.0,9.0,2.0,14.62]|   14.62|
|[3.0,10.0,5.0,957...|957.5775|
|[3.0,4.0,2.0,22.368]|  22.368|
| [0.0,2.0,7.0,48.86]|   48.86|
|  [0.0,5.0,4.0,7.28]|    7.28|
|[0.0,3.0,6.0,907....| 907.152|
|[0.0,0.0,3.0,18.504]|  18.504|
| [0.0,8.0,5.0,114.9]|   114.9|
|[0.0,10.0,9.0,170...|1706.184|
|[0.0,3.0,4.0,911....| 911.424|
|[3.0,1.0,3.0,15.552]|  15.552|
|[0.0,0.0,3.0,407....| 407.976|
| [2.0,8.0,5.0,68.81]|   68.81|
| [2.0,0.0,3.0,2.544]|   2.544|
|[2.0,4.0,6.0,665.88]|  665.88|
|  [0.0,4.0,2.0,55.5]|    55.5|
|  [0.0,5.0,2.0,8.56]|    8.56|
|[0.0,3.0,3.0,213.48]|  213.48|
+--------------------+--------+
only showing top 20 rows



In [15]:
train_data, test_data = final_data.randomSplit([0.80, 0.20])
regressor=LinearRegression (featuresCol='Independent Features', labelCol='sales')
regressor = regressor.fit(train_data)

In [16]:
predict_results = regressor.evaluate(test_data)

predict_results. predictions.show()

+--------------------+------+------------------+
|Independent Features| sales|        prediction|
+--------------------+------+------------------+
| [0.0,0.0,1.0,2.025]| 2.025| 2.024999999999784|
| [0.0,0.0,1.0,2.688]| 2.688|2.6879999999997843|
| [0.0,0.0,1.0,3.856]| 3.856|3.8559999999997845|
| [0.0,0.0,1.0,5.022]| 5.022| 5.021999999999785|
| [0.0,0.0,1.0,6.384]| 6.384| 6.383999999999786|
|[0.0,0.0,1.0,11.744]|11.744|11.743999999999785|
|[0.0,0.0,1.0,13.184]|13.184|13.183999999999786|
| [0.0,0.0,2.0,1.188]| 1.188|1.1879999999998077|
| [0.0,0.0,2.0,2.388]| 2.388| 2.387999999999808|
| [0.0,0.0,2.0,5.214]| 5.214| 5.213999999999809|
| [0.0,0.0,2.0,8.288]| 8.288|  8.28799999999981|
| [0.0,0.0,2.0,9.762]| 9.762|  9.76199999999981|
| [0.0,0.0,2.0,13.76]| 13.76|13.759999999999811|
|[0.0,0.0,2.0,13.904]|13.904|13.903999999999812|
|[0.0,0.0,2.0,14.624]|14.624|14.623999999999812|
|[0.0,0.0,2.0,17.456]|17.456|17.455999999999808|
|[0.0,0.0,2.0,17.568]|17.568| 17.56799999999981|
|[0.0,0.0,2.0,22.848

In [17]:
print("MAE is ", predict_results.meanAbsoluteError)
print("MSE is ", predict_results.meanSquaredError)
print("RMSE is ",predict_results.rootMeanSquaredError)
print("R2 is ",predict_results.r2)
print("Adj R2 is ", predict_results.r2adj)

MAE is  1.0070973187141372e-13
MSE is  2.6394568505977094e-26
RMSE is  1.6246405296550095e-13
R2 is  1.0
Adj R2 is  1.0
